# **Acquiring Data** for **My Spotify Library / Liked Songs**

# Imports

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import requests
from furl import furl
from math import ceil
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import sys
import pprint
import time

pp = pprint.PrettyPrinter()
%config InlineBackend.figure_format ='retina'

### Setting Global Variables:

In [2]:
SPOTIFY_CLIENT_ID='MY_CLIENT_ID'
SPOTIFY_CLIENT_SECRET='MY_CLIENT_SECRET'
SPOTIFY_REDIRECT_URI='MY_REDIRECT_URI'

## **Getting Tracks** From **My Library**:

In [3]:
username = 'MY_USERNAME'
scope = 'user-read-private user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username, 
                                   scope,
                                  client_id=SPOTIFY_CLIENT_ID,
                                  client_secret=SPOTIFY_CLIENT_SECRET,
                                  redirect_uri=SPOTIFY_REDIRECT_URI)

all_saved_tracks = {}

if token:
    sp = spotipy.Spotify(auth=token)
    for i in range(100):
        results  = sp.current_user_saved_tracks(limit=50, offset=(i*50))
        for item in results['items']:
            track = item['track']
#            track_info = {track['name']: [track['artists'][0]['name'], track['album']['name']]}
            all_saved_tracks[track['id']] = [track['name'], # track title
                                             track['artists'][0]['name'], # artist name
                                             track['artists'][0]['id'], # artist id
                                             track['album']['name'], # album title
                                             track['album']['id'], # album id
                                             track['album']['release_date'], # date released 
                                             track['popularity'],
                                             item['added_at']] # time added 
#             track_info = {track['name']: [track['artists'][0]['name'], track['album']['name']]}
#             all_saved_tracks[track['name']] = [track['artists'][0]['name'], track['album']['name']]
#            print(track['name'] + ' - ' + track['artists'][0]['name'])
else:
    print("Can't get token for", username)

### Putting Tracks Into a DataFrame

In [4]:
likedtracks_info = pd.DataFrame.from_dict(all_saved_tracks, orient='index', 
                                                  columns=['title', 
                                                           'artist', 
                                                           'artist_id', 
                                                           'album', 
                                                           'album_id', 
                                                           'album_releasedate',
                                                           'popularity',
                                                           'time_added']).reset_index()

In [5]:
likedtracks_info.rename({"index": "track_id"}, axis=1, inplace=True)

In [6]:
likedtracks_info.head()

,track_id,title,artist,artist_id,album,album_id,album_releasedate,popularity,time_added
0,0jt6NEuzEY0OouXkwZHq5n,Creepin Up The Backstairs - Composite Edit (do...,The Fratellis,3M4ThdJR28z9eSMcQHAZ5G,Costello Music,6YYhMNDgi37WNGtCDis6PT,2006-01-01,44,2020-06-09T21:02:29Z
1,1GAHIJ5a6AFbIW5yjrGdAz,Not Discuss It,St. Beauty,4j5EnP4t2lSKolLuQh1xvn,Running to the Sun,5TWAPPRs8ycFPPIEiXluZM,2018-01-19,30,2020-06-09T07:50:30Z
2,0QrxeCKmq1qHJSqysK5rdD,Drinking Song,Haley Heynderickx,73MDShZzdL4vUGMkmXOG6X,Fish Eyes EP,0AJs8uGiBASbgSV5t34yhp,2016-01-29,35,2020-06-07T06:45:48Z
3,1oG2vWELiGjIqxwqGcyqwF,You Really Got Me,The Kinks,1SQRv42e4PjEYfPhS0Tk9E,Kinks (Super Deluxe Edition),3hXXoaAA09avxBSSxtPIaL,1964-10-02,4,2020-06-06T21:30:23Z
4,78HAgo3mrVdg9sI9GVKfs5,Worlds to Run,Busdriver,43SXnO2N4X024u3l9MfvXD,Thumbs,7Dab5Lu2RVndW3c5W0xA1l,2015-11-06,51,2020-06-03T04:48:33Z


In [7]:
likedtracks_info.dtypes

track_id             object
title                object
artist               object
artist_id            object
album                object
album_id             object
album_releasedate    object
popularity            int64
time_added           object
dtype: object

#### Formatting time_added as DateTime, Splitting into numerical feature columns

In [8]:
likedtracks_info['full_time_added'] =  pd.to_datetime(likedtracks_info['time_added'], 
                                                         infer_datetime_format=True)



In [9]:
# likedtracks_info['date_added'] = likedtracks_info['full_time_added'].dt.date
# likedtracks_info['time_added'] = likedtracks_info['full_time_added'].dt.time
likedtracks_info['year_added'] = likedtracks_info['full_time_added'].dt.year
likedtracks_info['month_added'] = likedtracks_info['full_time_added'].dt.month
likedtracks_info['dayofweek_added'] = likedtracks_info['full_time_added'].dt.dayofweek
likedtracks_info['hour_added'] = likedtracks_info['full_time_added'].dt.hour

In [10]:
likedtracks_info.columns

Index(['track_id', 'title', 'artist', 'artist_id', 'album', 'album_id',
       'album_releasedate', 'popularity', 'time_added', 'full_time_added',
       'year_added', 'month_added', 'dayofweek_added', 'hour_added'],
      dtype='object')

In [11]:
likedtracks_info.dtypes

track_id                     object
title                        object
artist                       object
artist_id                    object
album                        object
album_id                     object
album_releasedate            object
popularity                    int64
time_added                   object
full_time_added      datetime64[ns]
year_added                    int64
month_added                   int64
dayofweek_added               int64
hour_added                    int64
dtype: object

#### Saving to CSV:

In [13]:
likedtracks_info.to_csv('01-Data--My-Library/likedtracks_info.csv')

---

# getting **Audio Features** for tracks:

In [14]:
# List of tracks used to create audio features dataframe
track_id_list = likedtracks_info['track_id'].to_list()

In [15]:
username = 'MY_USERNAME'
scope = 'user-read-private user-read-email user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username, 
                                   scope,
                                  client_id=SPOTIFY_CLIENT_ID,
                                  client_secret=SPOTIFY_CLIENT_SECRET,
                                  redirect_uri=SPOTIFY_REDIRECT_URI)

likedtracks_features = []

if token:
    sp = spotipy.Spotify(auth=token)
    for i in range(33):
        begin = i*100
        end = i*100 + 99
        track_master_list = track_id_list[begin:end+1]
        list_features = sp.audio_features(track_master_list)
        likedtracks_features.extend(list_features)

In [16]:
likedtracks_features = pd.DataFrame(likedtracks_features)
likedtracks_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.584,0.967,11,-4.069,0,0.2360,0.000302,0.000002,0.214,0.240,122.283,audio_features,0jt6NEuzEY0OouXkwZHq5n,spotify:track:0jt6NEuzEY0OouXkwZHq5n,https://api.spotify.com/v1/tracks/0jt6NEuzEY0O...,https://api.spotify.com/v1/audio-analysis/0jt6...,187160,4
1,0.567,0.392,1,-9.436,1,0.0419,0.087200,0.000049,0.112,0.195,89.976,audio_features,1GAHIJ5a6AFbIW5yjrGdAz,spotify:track:1GAHIJ5a6AFbIW5yjrGdAz,https://api.spotify.com/v1/tracks/1GAHIJ5a6AFb...,https://api.spotify.com/v1/audio-analysis/1GAH...,204310,4
2,0.544,0.539,0,-5.090,1,0.0292,0.088200,0.000000,0.142,0.237,120.877,audio_features,0QrxeCKmq1qHJSqysK5rdD,spotify:track:0QrxeCKmq1qHJSqysK5rdD,https://api.spotify.com/v1/tracks/0QrxeCKmq1qH...,https://api.spotify.com/v1/audio-analysis/0Qrx...,194808,3
3,0.567,0.959,8,-3.822,1,0.1020,0.099500,0.000061,0.100,0.961,137.377,audio_features,1oG2vWELiGjIqxwqGcyqwF,spotify:track:1oG2vWELiGjIqxwqGcyqwF,https://api.spotify.com/v1/tracks/1oG2vWELiGjI...,https://api.spotify.com/v1/audio-analysis/1oG2...,133933,4
4,0.618,0.670,4,-8.034,0,0.1500,0.136000,0.000116,0.169,0.214,143.059,audio_features,78HAgo3mrVdg9sI9GVKfs5,spotify:track:78HAgo3mrVdg9sI9GVKfs5,https://api.spotify.com/v1/tracks/78HAgo3mrVdg...,https://api.spotify.com/v1/audio-analysis/78HA...,315565,4


In [17]:
likedtracks_features.dtypes

danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
type                 object
id                   object
uri                  object
track_href           object
analysis_url         object
duration_ms           int64
time_signature        int64
dtype: object

#### Saving to CSV:

In [18]:
likedtracks_features.to_csv('01-Data--My-Library/likedtracks_features.csv')

### Combining Track Info with Audio Features:

In [19]:
likedtracks_info_audiofeatures = likedtracks_info.merge(likedtracks_features, left_on='track_id', right_on='id')

In [20]:
likedtracks_info_audiofeatures.to_csv('01-Data--My-Library/likedtracks_info_audiofeatures.csv')

---

### getting **Audio Analysis** for tracks:

### **NOTE**:

- I abandoned this because
    - this endpoint provides much more detailed information than I know how to use right now
    - this endpoint can only be accessed one track at a time because it returns so much information:
        - **multiple** lines of output per **second** of a track

In [21]:
# track_id_list[:10]

In [22]:
# username = 'MY_USERNAME'
# scope = 'user-read-private user-read-email user-library-read'

# if len(sys.argv) > 1:
#     username = sys.argv[1]
# else:
#     print("Usage: %s username" % (sys.argv[0],))
#     sys.exit()

# token = util.prompt_for_user_token(username, 
#                                    scope,
#                                   client_id=SPOTIFY_CLIENT_ID,
#                                   client_secret=SPOTIFY_CLIENT_SECRET,
#                                   redirect_uri=SPOTIFY_REDIRECT_URI)

# hannah_liked_tracks_audio_analysis = []

# number_of_tracks = likedtracks_info.shape[0]

# if token:
#     sp = spotipy.Spotify(auth=token)
# #     for each in track_id_list[:1]:
#     analysis = sp.audio_analysis('0QrxeCKmq1qHJSqysK5rdD')
#     print(analysis)
# #         hannah_liked_tracks_audio_analysis.extend(analysis)

#### Saving to CSV:

In [23]:
# hannah_liked_tracks_audio_analysis_df.to_csv('01-Data--My-Library/hannah_liked_tracks_audio_analysis_df.csv')

# Pulling **Genres** For Artists:

In [24]:
artist_id_list = likedtracks_info['artist_id'].to_list()
len(artist_id_list)

3305

In [25]:
artist_id_set = set(artist_id_list)
artist_id_unique_list = list(artist_id_set)
len(artist_id_unique_list)

565

In [26]:
for i in range(1):
        begin = i*50
        end = i*50 + 49
        artist_master_list = artist_id_list[begin:end+1]
len(set(artist_master_list))

30

In [27]:
username = 'MY_USERNAME'
scope = None

hannah_liked_tracks_artist_genres = {}

token = util.prompt_for_user_token(username, 
                                   scope,
                                  client_id=SPOTIFY_CLIENT_ID,
                                  client_secret=SPOTIFY_CLIENT_SECRET,
                                  redirect_uri=SPOTIFY_REDIRECT_URI)

if token:
    sp = spotipy.Spotify(auth=token)
    for i in range(12):
        begin = i*50
        end = i*50 + 49
        artist_master_list = artist_id_unique_list[begin:end+1]
        artist_info = sp.artists(artist_master_list)
        for item in artist_info['artists']:
            genres = item['genres']
            hannah_liked_tracks_artist_genres[item['id']] = str(genres)
#        pp.pprint(artist_info)
#         artist_genres = artist_info['genres']
#        hannah_liked_tracks_artist_genres.extend(artist_genres)

In [28]:
len(hannah_liked_tracks_artist_genres)

565

In [29]:
pp.pprint(hannah_liked_tracks_artist_genres)

{'00BueEoowSUVGZoCR5yIRL': "['broadway']",
 '01hzPWkdzFsejaSckW7JYB': "['old-time fiddle']",
 '02YLJJnWC7YQVixkjEBRn7': "['british singer-songwriter', 'chamber pop', "
                           "'indie anthem-folk', 'indie folk', 'indie pop']",
 '04RtvnUz0XrUbUZzOA8ZoW': "['emo', 'neon pop punk', 'pop punk', 'screamo']",
 '04dZCuDV1SdIZpqIKg7mrq': "['rhode island indie']",
 '053q0ukIDRgzwTr4vNSwab': "['art pop', 'canadian electropop', 'chillwave', "
                           "'dance pop', 'electropop', 'escape room', 'grave "
                           "wave', 'indietronica', 'metropopolis', 'new rave']",
 '054P3YpUCLmd6a8HwdWDXE': "['anarcho-punk', 'art punk', 'crust punk', "
                           "'hardcore punk', 'punk', 'thrash core']",
 '05SdqPzK4m3k1ljK2wrTSP': "['emo rap', 'pop rap']",
 '05fo024EFotg9songSENOZ': "['art pop', 'electronica', 'folktronica', 'freak "
                           "folk', 'new weird america']",
 '066X20Nz7iquqkkCW6Jxy6': "['alternative dance', 'a

In [30]:
hannah_liked_tracks_artist_genres_df = pd.DataFrame.from_dict(hannah_liked_tracks_artist_genres, orient='index').reset_index()

In [31]:
hannah_liked_tracks_artist_genres_df.rename({'index': 'artist_id', 0: 'genres'}, axis=1, inplace=True)

In [32]:
hannah_liked_tracks_artist_genres_df.head()

,artist_id,genres
0,3MdG05syQeRYPPcClLaUGl,"['adult standards', 'disco', 'funk', 'mellow g..."
1,2lTlrNNT915CUPJQlFOGVL,['anti-folk']
2,2S9W9aSAd7e5mp8WqWxN2h,"['dance pop', 'europop', 'pop', 'post-teen pop']"
3,3g2kUQ6tHLLbmkV7T4GPtL,"['art pop', 'chamber pop', 'indie pop', 'lilit..."
4,4ZyVcJrlUqF34lHnBwn3mK,['indie punk']


### Getting genres into a useable **list** format

In [33]:
hannah_liked_tracks_artist_genres.values()

dict_values(["['adult standards', 'disco', 'funk', 'mellow gold', 'motown', 'quiet storm', 'soft rock', 'soul']", "['anti-folk']", "['dance pop', 'europop', 'pop', 'post-teen pop']", "['art pop', 'chamber pop', 'indie pop', 'lilith', 'permanent wave', 'piano rock', 'pop rock', 'singer-songwriter']", "['indie punk']", "['alternative metal', 'christian alternative rock', 'christian rock', 'nu metal', 'pop emo', 'post-grunge']", "['broadway']", "['alternative dance', 'alternative rock', 'austindie', 'dance-punk', 'indie pop', 'indie rock', 'modern alternative rock', 'modern rock', 'new rave', 'rock']", "['dreamo']", "['minneapolis indie']", "['dance pop', 'pop', 'urban contemporary']", "['art pop', 'brooklyn indie', 'chillwave', 'experimental pop', 'freak folk', 'indie folk', 'indie pop', 'indie rock', 'indietronica']", "['shamanic']", "['pop']", "['progressive bluegrass']", "['hollywood', 'show tunes']", "['spirituals', 'vocal harmony group']", "['alternative r&b', 'experimental pop']", 

In [34]:
genres_string = str(hannah_liked_tracks_artist_genres.values())

In [35]:
punct_dict = {'[': '',
              ']': '',
              '"': ''}

In [36]:
genres_string = genres_string.lower()

In [37]:
trans = str.maketrans(punct_dict)

In [38]:
genres_string.translate(trans)

"dict_values('adult standards', 'disco', 'funk', 'mellow gold', 'motown', 'quiet storm', 'soft rock', 'soul', 'anti-folk', 'dance pop', 'europop', 'pop', 'post-teen pop', 'art pop', 'chamber pop', 'indie pop', 'lilith', 'permanent wave', 'piano rock', 'pop rock', 'singer-songwriter', 'indie punk', 'alternative metal', 'christian alternative rock', 'christian rock', 'nu metal', 'pop emo', 'post-grunge', 'broadway', 'alternative dance', 'alternative rock', 'austindie', 'dance-punk', 'indie pop', 'indie rock', 'modern alternative rock', 'modern rock', 'new rave', 'rock', 'dreamo', 'minneapolis indie', 'dance pop', 'pop', 'urban contemporary', 'art pop', 'brooklyn indie', 'chillwave', 'experimental pop', 'freak folk', 'indie folk', 'indie pop', 'indie rock', 'indietronica', 'shamanic', 'pop', 'progressive bluegrass', 'hollywood', 'show tunes', 'spirituals', 'vocal harmony group', 'alternative r&b', 'experimental pop', 'acoustic pop', 'indiecoustica', 'neo mellow', 'new americana', 'stomp a

In [39]:
genres2 = ['progressive bluegrass', 'contemporary country', 'alternative emo', 'canadian indie', 'canadian pop punk', 'canadian punk', 'indie punk', 'indie rock', 'toronto indie', 'canadian indie', 'chamber pop', 'indie folk', 'indie pop', 'new americana', 'stomp and holler', 'alternative rock', 'art rock', 'blues rock', 'dream pop', 'experimental rock', 'garage rock', 'indie rock', 'lo-fi', 'modern rock', 'new wave', 'noise pop', 'noise rock', 'post-punk', 'punk', 'rock', 'alternative rock', 'funk metal', 'funk rock', 'permanent wave', 'post-grunge', 'rock', 'barbadian pop', 'dance pop', 'pop', 'post-teen pop', 'r&b', 'urban contemporary', 'shimmer pop', 'indie pop', 'modern alternative rock', 'modern rock', 'nu gaze', 'pop', 'rock', 'alternative r&b', 'art pop', 'chamber pop', 'escape room', 'indie r&b', 'indie soul', 'la pop', 'indie folk', 'stomp and holler', 'swedish americana', 'swedish country', 'dance pop', 'dirty south rap', 'hip hop', 'miami hip hop', 'pop', 'pop rap', 'rap', 'southern hip hop', 'trap', 'urban contemporary', 'avant-garde metal', 'breakcore', 'drill and bass', 'french metal', 'chamber pop', 'indie folk', 'indie pop', 'new americana', 'stomp and holler', 'uk americana', 'dayton indie', 'olympia wa indie', 'riot grrrl', 'chamber pop', 'indie folk', 'alternative dance', 'chillwave', 'dream pop', 'electropop', 'indie pop', 'indie rock', 'indietronica', 'metropopolis', 'modern rock', 'shimmer pop', 'vapor soul', 'south carolina indie', 'conscious hip hop', 'hip hop', 'rap', 'west coast rap', '', 'adult standards', 'torch song', 'vocal jazz', 'alternative rock', 'chicago indie', 'dance-punk', 'garage rock', 'indie pop', 'indie rock', 'modern alternative rock', 'modern rock', 'permanent wave', 'pop rock', 'rock', 'christian rock', 'dreamo', 'indie pop', 'piano rock', '', 'canadian americana', 'folk', 'indie folk', 'new americana', 'art pop', 'australian indie', 'chamber pop', 'indie pop', 'indie rock', 'melbourne indie', 'dance pop', 'pop', 'post-teen pop', 'alternative dance', 'art pop', 'brooklyn indie', 'dance-punk', 'electropop', 'escape room', 'experimental pop', 'indie pop', 'indie rock', 'indie surf', 'indietronica', 'modern rock', 'new rave', 'noise pop', 'shimmer pop', 'alternative dance', 'art pop', 'dance pop', 'electropop', 'indietronica', 'neo-synthpop', 'new rave', 'alternative rock', 'anti-folk', 'baroque pop', 'canadian indie', 'chamber pop', 'indie folk', 'indie pop', 'indie rock', 'modern folk rock', 'modern rock', 'stomp and holler', 'charlotte nc indie', 'folk punk', '', 'emo', 'pop punk', 'screamo', 'anti-folk', 'indie punk', 'indie rock', 'small room', 'transpop', 'celtic rock', 'neo mellow', 'pop', 'pop rock', 'post-teen pop', 'viral pop', 'album rock', 'art rock', 'british invasion', 'classic rock', 'dance rock', 'folk rock', 'glam rock', 'mellow gold', 'protopunk', 'psychedelic rock', 'rock', 'roots rock', 'acoustic pop', 'british singer-songwriter', 'indie anthem-folk', 'indie folk', 'irish singer-songwriter', 'neo mellow', 'uk americana', 'pop', 'pop rap', 'rap', 'adult standards', 'jazz blues', 'lounge', 'soul', 'torch song', 'vocal jazz', 'brostep', 'dubstep', 'electronic trap', 'filthstep', 'zapstep', 'swedish soul', '', 'bluegrass', 'jam band', 'jamgrass', 'mandolin', 'old-time', 'progressive bluegrass', 'british singer-songwriter', 'chamber pop', 'indie anthem-folk', 'indie folk', 'indie pop', 'acoustic pop', 'indie folk', 'indiecoustica', 'neo mellow', 'new americana', 'stomp and holler', 'american folk revival', 'appalachian folk', 'banjo', 'bluegrass', 'old-time', 'string band', 'traditional folk', 'pop punk', 'post-hardcore', 'screamo', 'chicago rap', 'rap', 'alternative rock', 'blues rock', 'garage rock', 'modern blues rock', 'modern rock', 'permanent wave', 'punk blues', 'rock', 'country', 'folk', 'folk rock', 'lilith', 'mellow gold', 'pop rock', 'traditional folk', 'art pop', 'chamber pop', 'indie folk', 'indie pop', 'indie rock', 'alternative metal', 'permanent wave', 'post-grunge', 'punk', 'rock', 'skate punk', 'socal pop punk', 'dc indie', 'escape room', 'alternative r&b', 'experimental pop', 'dance pop', 'electropop', 'indie pop', 'metropopolis', 'pop', 'post-teen pop', 'uk alternative pop', 'emo', 'neon pop punk', 'pop punk', 'screamo', 'bluegrass', 'bluegrass gospel', 'folk', 'indie folk', 'new americana', 'progressive bluegrass', 'alternative rock', 'modern power pop', 'modern rock', 'permanent wave', 'pop rock', 'post-grunge', 'rock', 'alternative dance', 'art pop', 'dance pop', 'electronica', 'electropop', 'experimental pop', 'experimental vocal', 'icelandic experimental', 'icelandic pop', 'metropopolis', 'permanent wave', 'pop rock', 'trip hop', 'chamber pop', 'shimmer pop', 'chamber pop', 'indie folk', 'indie pop', 'indie rock', 'la indie', 'shimmer pop', 'stomp and holler', 'indietronica', 'vapor soul', 'appalachian folk', 'bluegrass', 'jam band', 'old-time', 'progressive bluegrass', 'acoustic punk', 'anti-folk', 'folk punk', 'alternative country', 'alternative rock', 'chamber pop', 'chicago indie', 'indie folk', 'indie pop', 'indie rock', 'rock', 'roots rock', 'alternative metal', 'alternative rock', 'cyberpunk', 'electronic rock', 'industrial', 'industrial metal', 'industrial rock', 'nu metal', 'post-grunge', 'rock', 'alternative metal', 'christian alternative rock', 'christian rock', 'nu metal', 'pop emo', 'post-grunge', 'canadian indie', 'chamber pop', 'indie pop', 'indie rock', 'indietronica', 'shimmer pop', 'stomp and holler', 'dance pop', 'electropop', 'indie pop', 'indie poptimism', 'pop', 'post-teen pop', 'swedish soul', 'alternative metal', 'electronic rock', 'electronica', 'industrial metal', 'industrial rock', 'alternative dance', 'alternative rock', 'art pop', 'dance-punk', 'garage rock', 'indie pop', 'indie rock', 'modern rock', 'new rave', 'rock', 'edm', 'electropop', 'indie poptimism', 'pop', 'tropical house', 'emo rap', 'pop rap', 'dance pop', 'pop', 'pop rap', 'indie pop', 'indie poptimism', 'modern alternative rock', 'modern rock', 'shimmer pop', '', 'rva indie', 'chamber pop', 'indie folk', 'indie pop', 'jacksonville indie', 'stomp and holler', 'french indie pop', 'indie folk', 'stomp and holler', 'deep new americana', 'indie folk', 'modern folk rock', 'new americana', 'portland indie', 'roots rock', 'stomp and holler', 'ambient folk', 'british singer-songwriter', 'chamber pop', 'indie anthem-folk', 'indie folk', 'broadway', 'hollywood', 'show tunes', 'la indie', 'neo mellow', 'pop rock', 'canadian hardcore', 'canadian pop punk', 'canadian punk', 'chaotic hardcore', 'indie punk', 'london on indie', 'bergen indie', 'melancholia', 'norwegian indie', 'pop punk', 'acoustic punk', 'folk punk', 'alternative rock', 'garage rock', 'modern rock', 'pop rock', 'post-grunge', 'rock', 'sacramento indie', 'indie punk', 'new jersey hardcore', 'new jersey indie', 'riot grrrl', 'chillhop', 'lo-fi house', 'art pop', 'british folk', 'british singer-songwriter', 'chamber pop', 'indie folk', 'singer-songwriter', 'stomp and holler', 'alternative metal', 'alternative rock', 'el paso indie', 'garage rock', 'modern rock', 'rock', 'alternative dance', 'alternative rock', 'baroque pop', 'canadian indie', 'chamber pop', 'dance-punk', 'garage rock', 'indie pop', 'indie rock', 'indietronica', 'modern rock', 'new rave', 'permanent wave', 'quebec indie', 'rock', 'album rock', 'art rock', 'blues rock', 'classic rock', 'folk rock', 'glam rock', 'hard rock', 'power pop', 'protopunk', 'psychedelic rock', 'pub rock', 'rock', 'roots rock', 'appalachian folk', 'bluegrass', 'instrumental bluegrass', 'old-time', 'progressive bluegrass', 'traditional folk', '', 'ambient', 'ambient pop', 'art pop', 'chamber pop', 'chillwave', 'compositional ambient', 'dream pop', 'drone', 'electra', 'experimental ambient', 'experimental pop', 'experimental rock', 'fourth world', 'freak folk', 'indie rock', 'lo-fi', 'modern dream pop', 'noise pop', 'noise rock', 'nu gaze', 'post-rock', 'shoegaze', 'slow core', 'warm drone', 'electronica', 'funk', 'indie jazz', 'indie soul', 'nu jazz', 'nz reggae', 'soul', 'alternative metal', 'alternative rock', 'blues rock', 'garage rock', 'modern rock', 'nu metal', 'palm desert scene', 'post-grunge', 'rock', 'stoner metal', 'stoner rock', 'american metalcore', 'mathcore', 'dance pop', 'europop', 'pop', 'post-teen pop', 'alternative dance', 'art pop', 'chillwave', 'dream pop', 'electronica', 'electropop', 'experimental pop', 'indie pop', 'indietronica', 'modern dream pop', 'neo-synthpop', 'new rave', 'alternative metal', 'alternative rock', 'christian rock', 'industrial metal', 'modern rock', 'nu metal', 'post-grunge', 'rap rock', 'rock', 'art pop', 'dance pop', 'electropop', 'indie pop', 'indietronica', 'pop', 'swedish electropop', 'swedish indie pop', 'swedish pop', 'acoustic pop', 'british singer-songwriter', 'chamber pop', 'indie folk', 'neo mellow', 'stomp and holler', 'dance-punk', 'garage rock', 'modern rock', 'new rave', 'folk-pop', 'escape room', 'minnesota hip hop', 'pop', 'pop rap', 'trap queen', 'folk punk', 'indie punk', 'indie rock', 'dance pop', 'pop', 'pop rap', 'post-teen pop', 'adult standards', 'classic soul', 'jazz blues', 'memphis soul', 'soul', 'southern soul', '', 'art pop', 'chamber pop', 'electronica', 'folktronica', 'freak folk', 'new weird america', 'anti-folk', 'electropop', 'piano rock', 'old-time fiddle', 'broadway', 'hollywood', 'show tunes', '', 'progressive bluegrass', 'electropop', 'escape room', 'hip pop', 'indie poptimism', 'indietronica', 'modern alternative rock', 'modern rock', 'pop', 'rock', 'big beat', 'disco house', 'alternative metal', 'emo', 'pop punk', 'post-hardcore', 'screamo', 'shamanic', 'emo', 'pop punk', 'escape room', 'chamber pop', 'indie folk', 'stomp and holler', 'stomp and holler', 'emo', 'neon pop punk', 'pop emo', 'pop punk', 'screamo', 'candy pop', 'dance rock', 'new romantic', 'new wave', 'new wave pop', 'permanent wave', 'power pop', 'rock', 'art pop', 'chamber pop', 'electropop', 'indie folk', 'indie pop', 'indie rock', 'small room', 'britpop', 'chamber pop', 'indie rock', 'modern rock', 'neo mellow', 'rock', 'bedroom pop', 'indie pop', 'modern rock', 'oakland indie', 'alternative dance', 'art pop', 'baroque pop', 'brooklyn indie', 'chamber pop', 'chillwave', 'dream pop', 'experimental pop', 'freak folk', 'indie folk', 'indie pop', 'indie rock', 'modern rock', 'new rave', 'new weird america', 'noise pop', 'nu gaze', 'shimmer pop', 'stomp and holler', 'irish rock', 'pop rock', 'adult standards', 'folk', 'lilith', 'neo mellow', 'vocal jazz', 'rhode island indie', 'acoustic pop', 'irish rock', 'irish singer-songwriter', 'neo mellow', 'singer-songwriter', 'indie punk', 'j-punk', 'japanese girl punk', 'japanese pop punk', 'japanese punk rock', 'electropop', 'indie electropop', 'indie poptimism', 'la pop', 'vapor soul', 'chamber pop', 'freak folk', 'indie folk', 'indie pop', 'new americana', 'north carolina indie', 'stomp and holler', 'vermont indie', 'permanent wave', 'punk', 'big band', 'jazz trumpet', 'new orleans jazz', 'stride', 'swing', 'vintage jazz', 'art pop', 'canadian indie', 'canadian pop', 'chamber pop', 'indie pop', 'indie rock', 'slow core', 'alternative rock', 'grunge', 'lilith', 'pop rock', 'post-grunge', 'rock', 'alternative rock', 'pop rock', 'alternative rock', 'melancholia', 'permanent wave', 'rock', 'singer-songwriter', '', 'alternative r&b', 'escape room', 'indie soul', 'lgbtq+ hip hop', 'acoustic punk', 'anti-folk', 'folk punk', 'texas punk', 'alternative dance', 'alternative r&b', 'art pop', 'chillwave', 'downtempo', 'electronica', 'electropop', 'escape room', 'gothenburg indie', 'indie soul', 'indietronica', 'new rave', 'swedish electropop', 'swedish synthpop', 'trip hop', 'indie folk', 'indie pop', 'indie rock', 'new jersey indie', 'small room', 'alternative dance', 'dance-punk', 'indie pop', 'indietronica', 'jacksonville indie', 'new rave', '', 'baroque pop', 'chamber pop', 'freak folk', 'indie folk', 'singer-songwriter', 'chamber pop', 'indie folk', 'indie pop', 'indie rock', 'modern rock', 'stomp and holler', 'alternative dance', 'art pop', 'brighton indie', 'chamber pop', 'chamber psych', 'dream pop', 'electronica', 'electropop', 'etherpop', 'experimental pop', 'indie pop', 'indie rock', 'indietronica', 'metropopolis', 'neo-synthpop', 'new rave', 'alternative dance', 'art pop', 'chillwave', 'experimental pop', 'indie pop', 'indie soul', 'indietronica', 'new rave', 'nu gaze', 'canadian americana', 'folk', 'art pop', 'canadian electropop', 'chillwave', 'edmonton indie', 'electropop', 'escape room', 'experimental pop', 'grave wave', 'indietronica', 'metropopolis', 'shimmer pop', 'alternative metal', 'alternative rock', 'garage rock', 'grunge', 'pop rock', 'post-grunge', 'rock', 'alternative rock', 'garage rock', 'indie rock', 'modern alternative rock', 'modern blues rock', 'modern rock', 'post-grunge', 'rock', 'supergroup', 'dance pop', 'electropop', 'pop', 'post-teen pop', 'alternative rock', 'blues rock', 'garage rock', 'modern blues rock', 'modern rock', 'punk blues', 'rock', 'dreamo', '', 'alternative country', 'country rock', 'folk', 'folk rock', 'indie folk', 'lilith', 'melancholia', 'nashville singer-songwriter', 'new americana', 'progressive bluegrass', 'roots rock', 'singer-songwriter', 'stomp and holler', 'traditional folk', 'alternative metal', 'alternative rock', 'modern rock', 'nu metal', 'permanent wave', 'post-grunge', 'rock', 'supergroup', 'downtempo', 'etherpop', 'lilith', 'metropopolis', 'bluegrass', 'bluegrass gospel', 'country gospel', 'album rock', 'classic rock', 'mellow gold', 'rock', 'soft rock', 'yacht rock', 'folk', 'indie folk', 'new americana', 'progressive bluegrass', 'stomp and holler', '', 'alternative rock', 'anti-folk', 'baroque pop', 'chamber pop', 'indie folk', 'indie pop', 'indie rock', 'melancholia', 'scottish indie', 'scottish rock', 'twee pop', 'emo rap', 'british soul', 'dance pop', 'europop', 'neo mellow', 'pop', 'post-teen pop', 'talent show', 'uk pop', 'urban contemporary', 'chamber pop', 'indie anthem-folk', 'indie folk', 'irish singer-songwriter', 'neo mellow', 'stomp and holler', 'dance pop', 'electropop', 'metropopolis', 'pop', 'post-teen pop', 'swedish electropop', 'swedish pop', 'swedish synthpop', 'modern rock', 'neo mellow', 'pop', 'pop punk', 'pop rock', 'post-grunge', 'dance pop', 'deep euro house', 'deep house', 'edm', 'german dance', 'pop', 'tropical house', 'dance pop', 'edm', 'electro house', 'moombahton', 'pop', 'pop rap', 'chamber pop', 'indie folk', 'new americana', 'progressive bluegrass', 'stomp and flutter', 'stomp and holler', 'hip hop', 'rap', 'alternative country', 'deep new americana', 'folk', 'indie folk', 'new americana', 'roots rock', 'stomp and holler', 'la indie', 'noise pop', 'nu gaze', 'anti-folk', 'lo-fi', '', 'alternative dance', 'alternative rock', 'austindie', 'dance-punk', 'garage rock', 'indie pop', 'indie rock', 'modern alternative rock', 'modern rock', 'rock', 'afrofuturism', 'alternative r&b', 'art pop', 'atl hip hop', 'dance pop', 'electropop', 'escape room', 'neo soul', 'pop', 'r&b', 'alternative rock', 'anti-folk', 'art pop', 'chamber pop', 'dream pop', 'freak folk', 'indie folk', 'indie pop', 'indie rock', 'lo-fi', 'new jersey indie', 'noise pop', 'permanent wave', 'slow core', 'modern rock', 'rock', 'alternative rock', 'anti-folk', 'athens indie', 'chamber pop', 'e6fi', 'freak folk', 'indie pop', 'indie rock', 'lo-fi', 'modern rock', 'noise pop', 'chamber pop', 'etherpop', 'alternative dance', 'big beat', 'breakbeat', 'electronica', 'new rave', 'rave', 'canadian rock', 'dance-punk', 'garage rock', 'indie rock', 'modern rock', 'new rave', 'victoria bc indie', 'art pop', 'dance pop', 'electropop', 'metropopolis', 'nz pop', 'pop', 'post-teen pop', 'bluegrass', 'folk', 'progressive bluegrass', 'portland indie', 'bluegrass', 'bluegrass gospel', '', 'pop', 'pop rock', 'canadian folk', 'canadian indie', 'canadian singer-songwriter', 'lilith', 'newfoundland indie', 'nl folk', '', 'cuban alternative', 'etherpop', 'electropop', 'indie pop', 'indie poptimism', 'indietronica', 'metropopolis', 'neo-synthpop', 'shimmer pop', 'anti-folk', 'canadian indie folk', 'small room', 'art pop', 'brooklyn indie', 'chamber pop', 'chillwave', 'experimental pop', 'freak folk', 'indie folk', 'indie pop', 'indie rock', 'indietronica', 'noise pop', 'escape room', 'hip pop', 'lgbtq+ hip hop', 'native american hip hop', 'transpop', 'trap queen', 'country rock', 'folk', 'folk rock', 'lilith', 'melancholia', 'mellow gold', 'roots rock', 'singer-songwriter', 'traditional folk', 'alternative metal', 'alternative rock', 'funk metal', 'funk rock', 'modern rock', 'nu metal', 'pop rock', 'post-grunge', 'rock', '', 'folk-pop', 'modern rock', 'stomp and holler', 'indie rock', 'modern rock', '', 'emo', 'neo mellow', 'pop punk', 'pop rock', 'pop folk', 'downtempo', 'dream pop', 'indietronica', 'adult standards', 'brill building pop', 'classic girl group', 'motown', 'soul', 'pop', 'neo mellow', 'neon pop punk', 'pop punk', 'pop rock', 'post-teen pop', 'vapor pop', 'psychedelic folk rock', 'neo soul', 'pop rock', 'soul', 'alternative country', 'canadian americana', 'folk', 'new americana', 'vancouver indie', 'art pop', 'chamber pop', 'electropop', 'etherpop', 'indie folk', 'indie pop', 'indie rock', 'indietronica', 'shimmer pop', 'bass trap', 'brostep', 'catstep', 'complextro', 'edm', 'electro house', 'electronic trap', 'filthstep', 'zapstep', '', 'stomp and holler', 'indie garage rock', 'neo-psychedelic', 'anti-folk', 'chamber pop', 'indie pop', 'lilith', 'piano rock', 'pop rock', 'acoustic pop', 'chamber pop', 'indie folk', 'indiecoustica', 'neo mellow', 'new americana', 'stomp and holler', 'emo', 'modern rock', 'pop punk', 'dance pop', 'electropop', 'indie pop', 'nyc pop', 'pop', 'austin americana', 'austindie', 'deep new americana', 'indie folk', 'modern rock', 'new americana', 'stomp and holler', 'piano rock', 'viral pop', 'alternative dance', 'alternative rock', 'chamber pop', 'dance-punk', 'garage rock', 'indie pop', 'indie rock', 'indietronica', 'la indie', 'modern rock', 'new rave', 'rock', 'shimmer pop', 'big beat', 'electronica', 'nu jazz', 'trip hop', 'orlando indie', 'acoustic pop', 'ectofolk', 'indie poptimism', 'indiecoustica', 'lilith', 'neo mellow', 'pop', 'pop rock', 'early us punk', 'protopunk', 'modern rock', 'rock', 'chillwave', 'experimental pop', 'new orleans indie', 'alternative rock', 'pop rock', 'post-grunge', 'alternative dance', 'dance-punk', 'indie pop', 'indie rock', 'indietronica', 'modern rock', 'new rave', 'afrofuturism', 'art pop', 'dance pop', 'electropop', 'escape room', 'indie r&b', 'trip hop', 'canadian pop', 'dance pop', 'electropop', 'indie poptimism', 'pop', 'post-teen pop', 'lgbtq+ hip hop', 'slam poetry', 'diy emo', 'indie punk', 'queercore', 'alternative metal', 'alternative rock', 'dance-punk', 'el paso indie', 'garage rock', 'indie rock', 'modern rock', 'post-hardcore', 'alternative rock', 'blues rock', 'garage rock', 'indie rock', 'modern blues rock', 'modern rock', 'punk blues', 'rock', '', 'alternative emo', 'emo', 'dance pop', 'hip pop', 'pop', 'pop rap', 'post-teen pop', 'queens hip hop', 'alternative rock', 'anti-folk', 'canadian indie', 'dance-punk', 'e6fi', 'experimental pop', 'freak folk', 'indie pop', 'indie rock', 'lo-fi', 'noise pop', 'duluth indie', 'stomp and holler', 'anti-folk', 'freak folk', 'indie pop', 'olympia wa indie', 'dreamo', 'emo', 'indie rock', 'philly indie', 'progressive post-hardcore', 'screamo', 'art pop', 'english indie rock', 'garage rock', 'indie rock', 'indietronica', 'modern alternative rock', 'modern rock', 'rock', 'adult standards', 'brill building pop', 'bubblegum pop', 'folk rock', 'rock-and-roll', 'rockabilly', 'indie punk', 'indie surf', 'power-pop punk', 'riot grrrl', 'seattle indie', '', 'metropopolis', 'modern rock', 'pop', 'pop rock', 'alternative emo', 'emo', 'grand rapids indie', 'pop punk', 'progressive post-hardcore', 'gangster rap', 'hip hop', 'pop rap', 'rap', 'southern hip hop', 'trap', 'underground hip hop', 'alternative emo', 'anti-folk', 'emo', 'folk punk', 'indie punk', 'indie rock', 'lo-fi', 'modern power pop', 'art pop', 'canadian electropop', 'canadian pop', 'dance pop', 'electropop', 'escape room', 'hyperpop', 'indie electropop', 'indie pop', 'indie poptimism', 'indietronica', 'metropopolis', 'pop', 'post-teen pop', 'vapor soul', 'indian folk', 'sitar', 'world', 'world fusion', 'alternative emo', 'emo', 'emo punk', 'indie punk', 'indie rock', 'midwest emo', 'small room', 'rap latina', 'trap queen', 'pop', 'appalachian folk', 'banjo', 'bluegrass', 'bluegrass gospel', 'old-time', 'progressive bluegrass', 'traditional folk', 'chamber pop', 'irish rock', 'irish singer-songwriter', 'melancholia', 'neo mellow', 'acoustic pop', 'alternative country', 'chamber pop', 'deep new americana', 'folk', 'indie folk', 'modern folk rock', 'neo mellow', 'new americana', 'progressive bluegrass', 'roots rock', 'stomp and holler', 'pop', 'alternative dance', 'electropop', 'escape room', 'indie pop', 'indietronica', 'new rave', 'boston punk', 'boston rock', 'punk', 'ska', 'ska punk', 'skate punk', 'saskatchewan indie', 'escape room', 'lgbtq+ hip hop', 'alternative country', 'alternative rock', 'dallas indie', 'folk', 'new americana', 'roots rock', 'anti-folk', 'dark cabaret', 'piano rock', 'steampunk', 'alternative emo', 'indie punk', 'melodic hardcore', 'acoustic punk', 'anti-folk', 'folk punk', 'indie rock', 'dance pop', 'edm', 'pop', 'pop rap', 'post-teen pop', 'tropical house', 'uk pop', '', 'alternative dance', 'canadian indie', 'canadian rock', 'dance-punk', 'electropop', 'garage rock', 'indie pop', 'indie rock', 'indietronica', 'lilith', 'metropopolis', 'modern rock', 'neo-synthpop', 'new rave', 'shimmer pop', 'folk punk', 'indie punk', 'melodic hardcore', 'olympia wa indie', 'orgcore', 'chamber pop', 'eau claire indie', 'indie folk', 'melancholia', 'slow core', 'contemporary vocal jazz', 'vocal jazz', 'chamber pop', 'indie folk', 'neo mellow', 'new americana', 'pop folk', 'stomp and flutter', 'stomp and holler', 'acoustic pop', 'folk', 'indie folk', 'lilith', 'modern folk rock', 'neo mellow', 'new americana', 'pop rock', 'stomp and holler', 'afrofuturism', 'alternative r&b', 'art pop', 'dance pop', 'electropop', 'escape room', 'hip pop', 'indie soul', 'neo soul', 'pop', 'r&b', 'urban contemporary', 'vocal harmony group', 'alternative country', 'anti-folk', 'art pop', 'canadian indie', 'canadian singer-songwriter', 'chamber pop', 'folk', 'indie folk', 'indie pop', 'indie rock', 'lilith', 'new americana', 'outlaw country', 'pop rock', 'roots rock', 'singer-songwriter', 'stomp and holler', 'traditional folk', 'alternative metal', 'neo mellow', 'pop rock', 'post-grunge', 'san marcos tx indie', 'hollywood', 'show tunes', 'indie folk', 'indie pop', 'indie rock', 'modern alternative rock', 'modern rock', 'stomp and holler', 'vapor soul', '', 'british soul', 'indie r&b', 'indie soul', 'neo soul', 'soul', 'alternative metal', 'canadian punk', 'nu metal', 'post-grunge', 'punk', 'british soul', 'dance pop', 'pop rock', 'alternative metal', 'alternative rock', 'funk metal', 'neo mellow', 'pop rock', 'post-grunge', 'big beat', 'bmore', 'downtempo', 'electronica', 'nu jazz', 'trip hop', 'chamber pop', 'wisconsin indie', 'alternative rock', 'art rock', 'classic rock', 'folk rock', 'melancholia', 'permanent wave', 'protopunk', 'psychedelic rock', 'rock', 'baroque pop', 'chicago indie', 'piano rock', 'big band', 'swing', 'alternative hip hop', 'art pop', 'appalachian folk', 'banjo', 'bluegrass', 'country rock', 'folk', 'jam band', 'old-time', 'progressive bluegrass', 'traditional folk', '', 'chicago rap', 'dance pop', 'dirty south rap', 'gangster rap', 'hip hop', 'pop rap', 'rap', 'trap', 'urban contemporary', 'alternative metal', 'alternative rock', 'pop rock', 'post-grunge', 'dance pop', 'europop', 'girl group', 'pop', 'pop rock', '', 'folk', 'indie folk', 'lilith', 'manitoba indie', 'new americana', 'progressive bluegrass', 'art pop', 'canadian electropop', 'chillwave', 'dance pop', 'electropop', 'escape room', 'grave wave', 'indietronica', 'metropopolis', '', 'experimental hip hop', 'milwaukee hip hop', 'appalachian folk', 'banjo', 'bluegrass', 'bluegrass gospel', 'country gospel', 'old-time', 'progressive bluegrass', 'traditional country', 'traditional folk', 'anti-folk', 'chamber pop', 'indie folk', 'indie pop', 'indie rock', 'melancholia', 'omaha indie', 'slow core', 'stomp and holler', '', '', 'dreamo', 'indie pop', 'indy indie', 'stomp and holler', 'australian garage punk', 'garage rock', 'indie garage rock', 'modern rock', 'noise pop', 'canadian pop', 'pop', 'post-teen pop', 'viral pop', 'canadian indie', 'indie pop', 'indie poptimism', 'indie rock', 'indietronica', 'lilith', 'metropopolis', 'permanent wave', 'pop rock', 'art pop', 'chamber pop', 'indie pop', 'lilith', 'permanent wave', 'piano rock', 'pop rock', 'singer-songwriter', 'alternative hip hop', 'conscious hip hop', 'minnesota hip hop', 'pop rap', 'nashville indie', 'riot grrrl', 'instrumental acoustic guitar', 'country blues', 'louisiana blues', 'new orleans blues', 'swing', 'alternative pop rock', 'neon pop punk', 'pop emo', 'pop punk', 'alternative country', 'deep new americana', 'folk', 'indie folk', 'new americana', 'garage rock', 'modern rock', 'permanent wave', 'rock', 'sheffield indie', 'anti-folk', 'art pop', 'chamber pop', 'freak folk', 'indie pop', 'indie rock', 'melancholia', 'singer-songwriter', 'slow core', 'dance pop', 'pop', 'post-teen pop', '', 'indie folk', 'memphis americana', 'new americana', 'atl hip hop', 'hip hop', 'pop rap', 'rap', 'alternative dance', 'art pop', 'dance-punk', 'electroclash', 'electronica', 'electropop', 'gothenburg indie', 'indietronica', 'neo-synthpop', 'new rave', 'swedish electropop', 'swedish synthpop', 'country rap', 'folk punk', 'emo', 'modern rock', 'neon pop punk', 'pop punk', 'atlanta indie', 'alternative rock', 'anti-folk', 'canadian indie', 'chamber pop', 'freak folk', 'indie folk', 'indie pop', 'indie rock', 'lo-fi', 'melancholia', 'new weird america', 'stomp and holler', '', 'acoustic punk', 'alternative emo', 'art pop', 'emo', 'folk punk', 'indie pop', 'indie punk', 'indie rock', 'philly indie', 'diy emo', 'canadian hip hop', 'canadian pop', 'hip hop', 'pop rap', 'rap', 'toronto rap', '', 'contemporary country', 'country', 'country dawn', 'anti-folk', 'boston rock', 'dark cabaret', 'piano rock', 'steampunk', 'dreamo', 'progressive post-hardcore', 'anti-folk', 'folk punk', 'indie punk', 'modern ska punk', 'ska', 'ska punk', 'acoustic punk', 'anti-folk', 'folk punk', 'lancaster pa indie', 'pittsburgh indie', 'anti-folk', 'indie folk', 'indie pop', 'indie rock', 'melancholia', 'omaha indie', 'stomp and holler', 'dance-punk', 'mathcore', 'nintendocore', 'post-hardcore', 'screamo', 'outlaw country', 'adult standards', 'nashville sound', 'electropop', 'indie poptimism', 'indietronica', 'modern alternative rock', 'modern rock', 'vapor soul', 'cali rap', 'gangster rap', 'hip hop', 'pop rap', 'rap', 'southern hip hop', 'trap', 'rhode island indie', 'social media pop', 'chamber pop', 'gothenburg indie', 'indie folk', 'neo mellow', 'stomp and holler', 'swedish indie folk', 'swedish indie pop', 'swedish singer-songwriter', 'latin jazz', 'world', 'emo', 'pop punk', '', '', 'acoustic punk', 'alternative rock', 'dance rock', 'punk', 'rock', 'dance pop', 'europop', 'pop', 'pop rap', 'post-teen pop', 'gauze pop', 'modern rock', 'shiver pop', 'indiana indie', 'bluegrass', 'folk', 'lilith', 'progressive bluegrass', 'art pop', 'alternative metal', 'canadian pop punk', 'canadian punk', 'nu metal', 'pop punk', 'post-grunge', 'punk', 'australian indie folk', 'chamber pop', 'indie anthem-folk', 'indie folk', 'neo mellow', 'stomp and holler', 'alternative dance', 'dance-punk', 'electroclash', 'new rave', 'queercore', 'riot grrrl', 'acoustic punk', 'anti-folk', 'folk punk', 'tucson indie', 'dreamo', 'progressive post-hardcore', 'screamo', 'comic', 'steampunk', 'alternative metal', 'alternative rock', 'permanent wave', 'pop rock', 'post-grunge', 'rock', 'escape room', 'alternative metal', 'nu metal', 'post-grunge', 'rap metal', 'rock', '', 'asheville indie', 'alternative rock', 'blues rock', 'post-grunge', 'rock', 'acoustic pop', 'indie folk', 'indiecoustica', 'neo mellow', 'pop rock', 'viral pop', 'electropop', 'gauze pop', 'indie pop', 'indie poptimism', 'indietronica', 'pop', 'adult standards', 'contemporary vocal jazz', 'neo mellow', 'vocal jazz', 'baltimore indie', 'indie punk', '', 'art rock', 'dance pop', 'dance rock', 'disco', 'electronica', 'new romantic', 'new wave', 'new wave pop', '', 'indie punk', 'indie rock', 'modern power pop', 'alternative dance', 'alternative rock', 'dance-punk', 'garage rock', 'indie rock', 'modern rock', 'new rave', 'rock', 'alternative country', 'folk', 'indie folk', 'new americana', 'new orleans americana', 'stomp and holler', 'traditional folk', 'anti-folk', 'dance-punk', 'indie pop', 'indie rock', 'new rave', 'welsh indie', 'welsh rock', 'emo', 'modern rock', 'pop punk', 'baroque pop', 'indie pop', 'indie rock', 'modern rock', 'bluegrass', 'pop punk', 'punk', 'socal pop punk', 'dance pop', 'neo mellow', 'pop', 'pop rock', 'post-teen pop', 'viral pop', 'alternative americana', 'art pop', 'chamber pop', 'chamber psych', 'dream pop', 'experimental pop', 'freak folk', 'indie pop', 'indie rock', 'alternative rock', 'art pop', 'chamber pop', 'chamber psych', 'dream pop', 'duluth indie', 'freak folk', 'indie rock', 'lo-fi', 'melancholia', 'post-rock', 'slow core', 'dance-punk', 'experimental rock', 'math rock', 'noise pop', 'noise rock', 'post-hardcore', 'uk noise rock', 'welsh rock', 'permanent wave', 'pop', 'baroque pop', 'emo', 'modern rock', 'country', 'country dawn', 'dance pop', 'pop rock', 'art pop', 'brooklyn indie', 'chamber pop', 'escape room', 'indie pop', 'indie rock', 'art pop', 'chamber pop', 'electropop', 'escape room', 'etherpop', 'experimental pop', 'indie pop', 'indie rock', 'indietronica', 'metropopolis', 'melancholia', 'modern alternative rock', 'modern rock', 'pop', 'pop rock', 'rock', 'stomp pop', 'indie punk', 'diy emo', 'indie punk', 'indie rock', 'alternative dance', 'alternative rock', 'dance-punk', 'garage rock', 'indie pop', 'indie rock', 'indietronica', 'modern rock', 'new rave', 'rock', 'alternative r&b', 'art pop', 'dance pop', 'electropop', 'indie poptimism', 'indie r&b', 'indietronica', 'pop', 'noise rock', 'candy pop', 'dance pop', 'lilith', 'neo mellow', 'pop rock', 'post-teen pop', 'vancouver indie', 'arkansas country', 'outlaw country', 'belgian pop', 'folk-pop', 'indie folk', 'indie pop', 'neo mellow', 'new americana', 'seattle indie', 'stomp and holler', 'lo-fi beats', 'indie folk', 'indie pop', 'new americana', 'stomp and holler', 'abstract hip hop', 'alternative hip hop', 'underground rap', 'alternative dance', 'art pop', 'dance pop', 'escape room', 'new rave', 'chamber folk', 'melancholia', 'alabama indie', 'omaha indie', 'anti-folk', 'british soul', 'neo soul', 'emo', 'modern power pop', 'modern rock', 'neo mellow', 'pop punk', 'pop rock', 'post-grunge', 'rock', 'dance pop', 'pop', 'post-teen pop', 'r&b', 'indie folk', 'new americana', 'north carolina indie', 'progressive bluegrass', 'roots rock', 'stomp and holler', 'east coast hip hop', 'hip hop', 'pop rap', 'rap', 'neo mellow', 'pop', 'pop rock', 'alternative dance', 'alternative rock', 'dance-punk', 'electronic rock', 'electronica', 'garage rock', 'indie pop', 'indie rock', 'indietronica', 'modern rock', 'new rave', 'rock', 'chillwave', 'electra', 'escape room', 'indie soul', 'indietronica', 'vapor soul', 'vapor twitch', 'wonky', 'dance pop', 'pop', 'urban contemporary', 'swedish alternative rock', 'swedish indie pop', '', 'art pop', 'art rock', 'baroque pop', 'dance rock', 'new wave pop', 'permanent wave', 'piano rock', 'pop rock', 'singer-songwriter', 'folk-pop', 'hollywood', 'scorecore', 'soundtrack', 'deep new americana', 'new americana', '', 'alternative hip hop', 'escape room', 'experimental hip hop', 'hip hop', 'rap', 'underground hip hop', '', 'neo mellow', 'art pop', 'chamber pop', 'chamber psych', 'dream pop', 'freak folk', 'indie folk', 'indie pop', 'indie rock', 'melancholia', 'new jersey indie', 'stomp and holler', 'garage rock', 'punk', 'skate punk', 'indie pop', 'modern alternative rock', 'modern rock', 'oakland indie', 'british singer-songwriter', 'chamber pop', 'fingerstyle', 'folk-pop', 'indie folk', 'neo mellow', 'stomp and holler', 'chattanooga indie', 'diy emo', 'midwest emo', 'alternative dance', 'alternative rock', 'dance pop', 'electronic rock', 'electronica', 'lilith', 'new wave pop', 'permanent wave', 'pop rock', 'post-grunge', 'rock', '', 'brooklyn indie', 'electropop', 'indie electropop', 'indie pop', 'indie poptimism', 'indietronica', 'nyc pop', 'pop', 'vapor soul', 'japanese math rock', 'japanese post-rock', '', 'alternative rock', 'emo', 'emo punk', 'indie punk', 'indie rock', 'lo-fi', 'melodic hardcore', 'orgcore', 'post-hardcore', 'punk', 'skate punk', 'indie pop', 'indie rock', 'modern rock', 'pop rock', 'washington indie', 'chamber pop', 'melancholia', 'norwegian singer-songwriter', 'chamber pop', 'indie folk', 'new americana', 'stomp and holler', 'neo mellow', 'piano rock', 'pop rock', 'banjo', 'clawhammer banjo', 'progressive bluegrass', 'r&b', 'chicago rap', 'conscious hip hop', 'hip hop', 'pop rap', 'rap', 'folk punk', 'olympia wa indie', 'queercore', 'riot grrrl', 'irish singer-songwriter', 'pop', 'alternative pop', 'alternative rock', 'anti-folk', 'art pop', 'indie pop', 'indie punk', 'indie rock', 'lilith', 'lo-fi', 'noise pop', 'olympia wa indie', 'riot grrrl', 'alternative dance', 'alternative rock', 'dance-punk', 'garage rock', 'indie rock', 'modern blues rock', 'modern rock', 'new rave', 'rock', 'gaian doom', 'ambient pop', 'downtempo', 'electronica', 'trip hop', '', 'alternative emo', 'emo', 'indie rock', 'modern rock', 'new jersey indie', 'pop punk', 'anti-folk', 'folk punk', 'olympia wa indie', 'neon pop punk', 'pop emo', 'pop punk', '', 'art pop', 'freak folk', 'gaian doom', 'sacramento indie', 'modern alternative rock', 'modern rock', 'rock', 'electropop', 'a cappella', 'college a cappella', 'desi pop', 'filmi', 'modern bollywood', 'pop punk', 'folk-pop', 'indie pop', 'metropopolis', 'modern rock', 'pop rock', 'stomp and holler', 'alternative country', 'folk', 'indie folk', 'neo mellow', 'new americana', 'pop rock', 'roots rock', 'singer-songwriter', 'stomp and holler', 'alternative rock', 'anti-folk', 'chamber pop', 'indie folk', 'indie pop', 'indie rock', 'lo-fi', 'melancholia', 'singer-songwriter', 'slow core', '', 'chamber pop', 'indie folk', 'stomp and holler', 'swedish americana', 'swedish singer-songwriter', '', 'chamber pop', 'freak folk', 'indie folk', 'indie pop', 'indie rock', 'modern rock', 'seattle indie', 'stomp and holler', 'folk', 'indie folk', 'lilith', 'new americana', 'progressive bluegrass', 'stomp and holler', 'dreamo', 'indie pop', 'indie rock', 'modern rock', 'minneapolis indie', 'classical', 'impressionism', 'post-romantic era', 'alternative emo', 'buffalo ny indie', 'indie punk', 'art pop', 'baroque pop', 'pop', 'uk alternative pop', 'alternative rock', 'art pop', 'art rock', 'chamber pop', 'chamber psych', 'garage rock', 'indie rock', 'lilith', 'melancholia', 'permanent wave', 'piano rock', 'rock', 'singer-songwriter', 'acoustic pop', 'christian indie', 'indie folk', 'indiecoustica', 'nashville americana', 'nashville singer-songwriter', 'neo mellow', 'new americana', 'art pop', 'electronica', 'laboratorio', 'trip hop', 'british folk', 'chamber pop', 'folk', 'folk rock', 'freak folk', 'indie folk', 'melancholia', 'rock', 'roots rock', 'singer-songwriter', 'slow core', 'dance pop', 'edm', 'gauze pop', 'pop', 'post-teen pop', 'tropical house', 'uk pop', 'candy pop', 'emo', 'pixie', 'pop emo', 'pop punk', 'british soul', 'neo mellow', 'neo soul', 'soul', 'afrofuturism', 'british soul', 'neo soul', 'hollywood', 'show tunes', 'alternative r&b', 'dance pop', 'deep pop r&b', 'electropop', 'hip pop', 'metropopolis', 'pop', 'pop rap', 'post-teen pop', 'r&b', 'urban contemporary', 'acoustic pop', 'hollywood', 'lilith', 'neo mellow', 'pop', 'pop rock', 'post-teen pop', 'alternative dance', 'alternative rock', 'baroque pop', 'canadian indie', 'canadian rock', 'chamber pop', 'chillwave', 'dance-punk', 'dream pop', 'experimental pop', 'freak folk', 'indie folk', 'indie pop', 'indie rock', 'lo-fi', 'modern rock', 'noise pop', 'nu gaze', 'post-rock', 'slow core', 'stomp and holler', '', 'austin singer-songwriter', 'art pop', 'chamber pop', 'escape room', 'experimental pop', 'freak folk', 'indie pop', 'indie rock', 'one-person band', '', 'indie garage rock', 'indie punk', 'indie rock', 'indie surf', 'lo-fi', 'noise pop', 'noise rock', 'nu gaze', 'san diego indie', 'surf punk', 'dance pop', 'pop', 'post-teen pop', 'albuquerque indie', 'alternative rock', 'chamber pop', 'garage rock', 'indie folk', 'indie pop', 'indie rock', 'modern rock', 'portland indie', 'stomp and holler', 'folk', 'indie folk', 'new americana', 'progressive bluegrass', 'indie punk', 'indie rock', 'philly indie', 'olympia wa indie', 'riot grrrl', '', 'candy pop', 'dance pop', 'electropop', 'metropopolis', 'pop', 'post-teen pop', 'tropical house', 'dance pop', 'electropop', 'etherpop', 'indie poptimism', 'pop', 'post-teen pop', 'bay area indie', 'chamber pop', 'freak folk', 'indie pop', 'indie rock', 'shimmer pop', 'stomp and holler', 'canadian indie', 'vancouver indie', '', 'pop', 'uk pop', 'anti-folk', 'folk punk', 'neo soul', 'pop', 'r&b', 'urban contemporary', 'alternative rock', 'dance-punk', 'garage rock', 'modern rock', 'new rave', 'rock', 'swedish garage rock', 'swedish hard rock', 'swedish indie rock', 'alternative dance', 'electronica', 'indie soul', 'indietronica', 'new rave', 'nu disco', 'british soul', 'pop', 'uk pop', 'anti-folk', 'buffalo ny indie', 'ectofolk', 'folk', 'lilith', 'pop rock', 'singer-songwriter', 'modern rock', 'pop punk', 'post-teen pop', 'electropop', 'indietronica', 'modern rock', 'shimmer pop', 'acoustic pop', 'chamber pop', 'indie folk', 'indie pop', 'neo mellow', 'stomp and holler', 'freak folk', 'philly indie', 'stomp and holler', 'chicano punk', 'power-pop punk', 'baroque pop', 'chamber pop', 'chicago indie', 'freak folk', 'indie folk', 'indie pop', 'indie rock', 'melancholia', 'stomp and holler', 'south carolina indie', 'alternative country', 'anti-folk', 'chamber pop', 'freak folk', 'indie folk', 'indie pop', 'new americana', 'portland indie', 'slow core', 'stomp and holler', 'indie pop', 'indie rock', 'indietronica', 'modern rock', 'seattle indie', 'classic soul', 'funk', 'memphis soul', 'quiet storm', 'soul', 'soul blues', 'southern soul', 'bristol indie', 'english indie rock', 'garage rock', 'modern alternative rock', 'modern rock', 'acoustic punk', 'anti-folk', 'folk punk', 'alternative emo', 'anthem emo', 'diy emo', 'emo', 'indie punk', 'indie rock', 'midwest emo', 'small room', 'emo', 'pop punk', 'pop rock', 'post-grunge', 'screamo', 'alternative rock', 'dance-punk', 'garage rock', 'indie pop', 'indie rock', 'modern rock', 'rock', 'washington indie', 'cleveland indie', 'diy emo', 'anarcho-punk', 'art punk', 'crust punk', 'hardcore punk', 'punk', 'thrash core', 'abstract hip hop', 'alternative hip hop', 'escape room', 'bow pop', 'emo', 'pop punk', 'pop rock', 'post-grunge', 'socal pop punk', 'austin americana', 'austindie', 'chamber pop', 'deep new americana', 'folk-pop', 'indie folk', 'stomp and holler', '', 'adult standards', 'disco', 'funk', 'mellow gold', 'motown', 'quiet storm', 'soft rock', 'soul', 'bluegrass', 'indie folk', 'instrumental bluegrass', 'mandolin', 'progressive bluegrass']

In [40]:
type(genres2)

list

In [41]:
genres = set(genres2)

In [42]:
genres

{'',
 'a cappella',
 'abstract hip hop',
 'acoustic pop',
 'acoustic punk',
 'adult standards',
 'afrofuturism',
 'alabama indie',
 'album rock',
 'albuquerque indie',
 'alternative americana',
 'alternative country',
 'alternative dance',
 'alternative emo',
 'alternative hip hop',
 'alternative metal',
 'alternative pop',
 'alternative pop rock',
 'alternative r&b',
 'alternative rock',
 'ambient',
 'ambient folk',
 'ambient pop',
 'american folk revival',
 'american metalcore',
 'anarcho-punk',
 'anthem emo',
 'anti-folk',
 'appalachian folk',
 'arkansas country',
 'art pop',
 'art punk',
 'art rock',
 'asheville indie',
 'athens indie',
 'atl hip hop',
 'atlanta indie',
 'austin americana',
 'austin singer-songwriter',
 'austindie',
 'australian garage punk',
 'australian indie',
 'australian indie folk',
 'avant-garde metal',
 'baltimore indie',
 'banjo',
 'barbadian pop',
 'baroque pop',
 'bass trap',
 'bay area indie',
 'bedroom pop',
 'belgian pop',
 'bergen indie',
 'big band'

In [43]:
genres_list = ['', 'a cappella', 'abstract hip hop', 'acoustic pop', 'acoustic punk', 'adult standards', 'afrofuturism', 'alabama indie', 'album rock', 'albuquerque indie', 'alternative americana', 'alternative country', 'alternative dance', 'alternative emo', 'alternative hip hop', 'alternative metal', 'alternative pop', 'alternative pop rock', 'alternative r&b', 'alternative rock', 'ambient', 'ambient folk', 'ambient pop', 'american folk revival', 'american metalcore', 'anarcho-punk', 'anthem emo', 'anti-folk', 'appalachian folk', 'arkansas country', 'art pop', 'art punk', 'art rock', 'asheville indie', 'athens indie', 'atl hip hop', 'atlanta indie', 'austin americana', 'austin singer-songwriter', 'austindie', 'australian garage punk', 'australian indie', 'australian indie folk', 'avant-garde metal', 'baltimore indie', 'banjo', 'barbadian pop', 'baroque pop', 'bass trap', 'bay area indie', 'bedroom pop', 'belgian pop', 'bergen indie', 'big band', 'big beat', 'bluegrass', 'bluegrass gospel', 'blues rock', 'bmore', 'boston punk', 'boston rock', 'bow pop', 'breakbeat', 'breakcore', 'brighton indie', 'brill building pop', 'bristol indie', 'british folk', 'british invasion', 'british singer-songwriter', 'british soul', 'britpop', 'broadway', 'brooklyn indie', 'brostep', 'bubblegum pop', 'buffalo ny indie', 'cali rap', 'canadian americana', 'canadian electropop', 'canadian folk', 'canadian hardcore', 'canadian hip hop', 'canadian indie', 'canadian indie folk', 'canadian pop', 'canadian pop punk', 'canadian punk', 'canadian rock', 'canadian singer-songwriter', 'candy pop', 'catstep', 'celtic rock', 'chamber folk', 'chamber pop', 'chamber psych', 'chaotic hardcore', 'charlotte nc indie', 'chattanooga indie', 'chicago indie', 'chicago rap', 'chicano punk', 'chillhop', 'chillwave', 'christian alternative rock', 'christian indie', 'christian rock', 'classic girl group', 'classic rock', 'classic soul', 'classical', 'clawhammer banjo', 'cleveland indie', 'college a cappella', 'comic', 'complextro', 'compositional ambient', 'conscious hip hop', 'contemporary country', 'contemporary vocal jazz', 'country', 'country blues', 'country dawn', 'country gospel', 'country rap', 'country rock', 'crust punk', 'cuban alternative', 'cyberpunk', 'dallas indie', 'dance pop', 'dance rock', 'dance-punk', 'dark cabaret', 'dayton indie', 'dc indie', 'deep euro house', 'deep house', 'deep new americana', 'deep pop r&b', 'desi pop', 'dirty south rap', 'disco', 'disco house', 'diy emo', 'downtempo', 'dream pop', 'dreamo', 'drill and bass', 'drone', 'dubstep', 'duluth indie', 'e6fi', 'early us punk', 'east coast hip hop', 'eau claire indie', 'ectofolk', 'edm', 'edmonton indie', 'el paso indie', 'electra', 'electro house', 'electroclash', 'electronic rock', 'electronic trap', 'electronica', 'electropop', 'emo', 'emo punk', 'emo rap', 'english indie rock', 'escape room', 'etherpop', 'europop', 'experimental ambient', 'experimental hip hop', 'experimental pop', 'experimental rock', 'experimental vocal', 'filmi', 'filthstep', 'fingerstyle', 'folk', 'folk punk', 'folk rock', 'folk-pop', 'folktronica', 'fourth world', 'freak folk', 'french indie pop', 'french metal', 'funk', 'funk metal', 'funk rock', 'gaian doom', 'gangster rap', 'garage rock', 'gauze pop', 'german dance', 'girl group', 'glam rock', 'gothenburg indie', 'grand rapids indie', 'grave wave', 'grunge', 'hard rock', 'hardcore punk', 'hip hop', 'hip pop', 'hollywood', 'hyperpop', 'icelandic experimental', 'icelandic pop', 'impressionism', 'indian folk', 'indiana indie', 'indie anthem-folk', 'indie electropop', 'indie folk', 'indie garage rock', 'indie jazz', 'indie pop', 'indie poptimism', 'indie punk', 'indie r&b', 'indie rock', 'indie soul', 'indie surf', 'indiecoustica', 'indietronica', 'industrial', 'industrial metal', 'industrial rock', 'indy indie', 'instrumental acoustic guitar', 'instrumental bluegrass', 'irish rock', 'irish singer-songwriter', 'j-punk', 'jacksonville indie', 'jam band', 'jamgrass', 'japanese girl punk', 'japanese math rock', 'japanese pop punk', 'japanese post-rock', 'japanese punk rock', 'jazz blues', 'jazz trumpet', 'la indie', 'la pop', 'laboratorio', 'lancaster pa indie', 'latin jazz', 'lgbtq+ hip hop', 'lilith', 'lo-fi', 'lo-fi beats', 'lo-fi house', 'london on indie', 'louisiana blues', 'lounge', 'mandolin', 'manitoba indie', 'math rock', 'mathcore', 'melancholia', 'melbourne indie', 'mellow gold', 'melodic hardcore', 'memphis americana', 'memphis soul', 'metropopolis', 'miami hip hop', 'midwest emo', 'milwaukee hip hop', 'minneapolis indie', 'minnesota hip hop', 'modern alternative rock', 'modern blues rock', 'modern bollywood', 'modern dream pop', 'modern folk rock', 'modern power pop', 'modern rock', 'modern ska punk', 'moombahton', 'motown', 'nashville americana', 'nashville indie', 'nashville singer-songwriter', 'nashville sound', 'native american hip hop', 'neo mellow', 'neo soul', 'neo-psychedelic', 'neo-synthpop', 'neon pop punk', 'new americana', 'new jersey hardcore', 'new jersey indie', 'new orleans americana', 'new orleans blues', 'new orleans indie', 'new orleans jazz', 'new rave', 'new romantic', 'new wave', 'new wave pop', 'new weird america', 'newfoundland indie', 'nintendocore', 'nl folk', 'noise pop', 'noise rock', 'north carolina indie', 'norwegian indie', 'norwegian singer-songwriter', 'nu disco', 'nu gaze', 'nu jazz', 'nu metal', 'nyc pop', 'nz pop', 'nz reggae', 'oakland indie', 'old-time', 'old-time fiddle', 'olympia wa indie', 'omaha indie', 'one-person band', 'orgcore', 'orlando indie', 'outlaw country', 'palm desert scene', 'permanent wave', 'philly indie', 'piano rock', 'pittsburgh indie', 'pixie', 'pop', 'pop emo', 'pop folk', 'pop punk', 'pop rap', 'pop rock', 'portland indie', 'post-grunge', 'post-hardcore', 'post-punk', 'post-rock', 'post-romantic era', 'post-teen pop', 'power pop', 'power-pop punk', 'progressive bluegrass', 'progressive post-hardcore', 'protopunk', 'psychedelic folk rock', 'psychedelic rock', 'pub rock', 'punk', 'punk blues', 'quebec indie', 'queens hip hop', 'queercore', 'quiet storm', 'r&b', 'rap', 'rap latina', 'rap metal', 'rap rock', 'rave', 'rhode island indie', 'riot grrrl', 'rock', 'rock-and-roll', 'rockabilly', 'roots rock', 'rva indie', 'sacramento indie', 'san diego indie', 'san marcos tx indie', 'saskatchewan indie', 'scorecore', 'scottish indie', 'scottish rock', 'screamo', 'seattle indie', 'shamanic', 'sheffield indie', 'shimmer pop', 'shiver pop', 'shoegaze', 'show tunes', 'singer-songwriter', 'sitar', 'ska', 'ska punk', 'skate punk', 'slam poetry', 'slow core', 'small room', 'socal pop punk', 'social media pop', 'soft rock', 'soul', 'soul blues', 'soundtrack', 'south carolina indie', 'southern hip hop', 'southern soul', 'steampunk', 'stomp and flutter', 'stomp and holler', 'stomp pop', 'stoner metal', 'stoner rock', 'stride', 'string band', 'supergroup', 'surf punk', 'swedish alternative rock', 'swedish americana', 'swedish country', 'swedish electropop', 'swedish garage rock', 'swedish hard rock', 'swedish indie folk', 'swedish indie pop', 'swedish indie rock', 'swedish pop', 'swedish singer-songwriter', 'swedish soul', 'swedish synthpop', 'swing', 'talent show', 'texas punk', 'thrash core', 'torch song', 'toronto indie', 'toronto rap', 'traditional country', 'traditional folk', 'transpop', 'trap', 'trap queen', 'trip hop', 'tropical house', 'tucson indie', 'twee pop', 'uk alternative pop', 'uk americana', 'uk noise rock', 'uk pop', 'underground hip hop', 'underground rap', 'urban contemporary', 'vancouver indie', 'vapor pop', 'vapor soul', 'vapor twitch', 'vermont indie', 'victoria bc indie', 'vintage jazz', 'viral pop', 'vocal harmony group', 'vocal jazz', 'warm drone', 'washington indie', 'welsh indie', 'welsh rock', 'west coast rap', 'wisconsin indie', 'wonky', 'world', 'world fusion', 'yacht rock', 'zapstep']

In [44]:
len(genres_list)

479

In [45]:
artist_genres_df = hannah_liked_tracks_artist_genres_df.copy(True)

In [46]:
artist_genres_df.rename({'index': 'artist_id', 0: 'genres'}, axis=1, inplace=True)

In [47]:
artist_genres_df.shape

(565, 2)

In [48]:
artist_genres_df.head()

,artist_id,genres
0,3MdG05syQeRYPPcClLaUGl,"['adult standards', 'disco', 'funk', 'mellow g..."
1,2lTlrNNT915CUPJQlFOGVL,['anti-folk']
2,2S9W9aSAd7e5mp8WqWxN2h,"['dance pop', 'europop', 'pop', 'post-teen pop']"
3,3g2kUQ6tHLLbmkV7T4GPtL,"['art pop', 'chamber pop', 'indie pop', 'lilit..."
4,4ZyVcJrlUqF34lHnBwn3mK,['indie punk']


## Creating a Dataframe For Genres as Categorical Values
- Iterated over genres and artists to create a binary dataframe for whether an artist is associated with each genre

In [49]:
artist_genres_categorical_df = artist_genres_df

In [50]:
for genre in genres_list:
    artist_genres_categorical_df[genre] = 0
    for i in range(559):
        if genre in artist_genres_categorical_df['genres'][i]:
            artist_genres_categorical_df[genre][i] += 1
        else:
            pass

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [51]:
artist_genres_categorical_df = artist_genres_categorical_df.set_index('artist_id').drop(['genres', ''], axis=1)

In [52]:
artist_genres_categorical_df.head()

,a cappella,abstract hip hop,acoustic pop,acoustic punk,adult standards,afrofuturism,alabama indie,album rock,albuquerque indie,alternative americana,...,washington indie,welsh indie,welsh rock,west coast rap,wisconsin indie,wonky,world,world fusion,yacht rock,zapstep
artist_id,,,,,,,,,,,,,,,,,,,,,
3MdG05syQeRYPPcClLaUGl,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2lTlrNNT915CUPJQlFOGVL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2S9W9aSAd7e5mp8WqWxN2h,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3g2kUQ6tHLLbmkV7T4GPtL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4ZyVcJrlUqF34lHnBwn3mK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Can we **merge** these bad boys?

In [53]:
likedtracks_info.head(1)

,track_id,title,artist,artist_id,album,album_id,album_releasedate,popularity,time_added,full_time_added,year_added,month_added,dayofweek_added,hour_added
0,0jt6NEuzEY0OouXkwZHq5n,Creepin Up The Backstairs - Composite Edit (do...,The Fratellis,3M4ThdJR28z9eSMcQHAZ5G,Costello Music,6YYhMNDgi37WNGtCDis6PT,2006-01-01,44,2020-06-09T21:02:29Z,2020-06-09 21:02:29,2020,6,1,21


In [54]:
likedtracks_info.set_index('artist_id', inplace=True)

In [55]:
likedtracks_info.columns

Index(['track_id', 'title', 'artist', 'album', 'album_id', 'album_releasedate',
       'popularity', 'time_added', 'full_time_added', 'year_added',
       'month_added', 'dayofweek_added', 'hour_added'],
      dtype='object')

In [56]:
hannah_liked_tracks_artist_genres_df.set_index('artist_id', inplace=True)

In [57]:
hannah_liked_tracks_artist_genres_df.head()

,genres
artist_id,
3MdG05syQeRYPPcClLaUGl,"['adult standards', 'disco', 'funk', 'mellow g..."
2lTlrNNT915CUPJQlFOGVL,['anti-folk']
2S9W9aSAd7e5mp8WqWxN2h,"['dance pop', 'europop', 'pop', 'post-teen pop']"
3g2kUQ6tHLLbmkV7T4GPtL,"['art pop', 'chamber pop', 'indie pop', 'lilit..."
4ZyVcJrlUqF34lHnBwn3mK,['indie punk']


In [58]:
likedtracks_info.head(1)

,track_id,title,artist,album,album_id,album_releasedate,popularity,time_added,full_time_added,year_added,month_added,dayofweek_added,hour_added
artist_id,,,,,,,,,,,,,
3M4ThdJR28z9eSMcQHAZ5G,0jt6NEuzEY0OouXkwZHq5n,Creepin Up The Backstairs - Composite Edit (do...,The Fratellis,Costello Music,6YYhMNDgi37WNGtCDis6PT,2006-01-01,44,2020-06-09T21:02:29Z,2020-06-09 21:02:29,2020,6,1,21


In [59]:
likedtracks_info_features_genres = likedtracks_info_audiofeatures.merge(hannah_liked_tracks_artist_genres_df, on='artist_id') #lsuffix='_tracks', rsuffix='_genres'

In [60]:
likedtracks_info_features_genres.to_csv('01-Data--My-Library/likedtracks_info_features_genres.csv')

In [61]:
hannah_tracks_and_genres = likedtracks_info_features_genres

In [62]:
hannah_tracks_and_genres.head(2)

,track_id,title,artist,artist_id,album,album_id,album_releasedate,popularity,time_added,full_time_added,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genres
0,0jt6NEuzEY0OouXkwZHq5n,Creepin Up The Backstairs - Composite Edit (do...,The Fratellis,3M4ThdJR28z9eSMcQHAZ5G,Costello Music,6YYhMNDgi37WNGtCDis6PT,2006-01-01,44,2020-06-09T21:02:29Z,2020-06-09 21:02:29,...,0.240,122.283,audio_features,0jt6NEuzEY0OouXkwZHq5n,spotify:track:0jt6NEuzEY0OouXkwZHq5n,https://api.spotify.com/v1/tracks/0jt6NEuzEY0O...,https://api.spotify.com/v1/audio-analysis/0jt6...,187160,4,"['alternative rock', 'garage rock', 'modern ro..."
1,1GAHIJ5a6AFbIW5yjrGdAz,Not Discuss It,St. Beauty,4j5EnP4t2lSKolLuQh1xvn,Running to the Sun,5TWAPPRs8ycFPPIEiXluZM,2018-01-19,30,2020-06-09T07:50:30Z,2020-06-09 07:50:30,...,0.195,89.976,audio_features,1GAHIJ5a6AFbIW5yjrGdAz,spotify:track:1GAHIJ5a6AFbIW5yjrGdAz,https://api.spotify.com/v1/tracks/1GAHIJ5a6AFb...,https://api.spotify.com/v1/audio-analysis/1GAH...,204310,4,[]


In [63]:
hannah_tracks_and_genres.shape

(3300, 33)

In [64]:
num_tracks = hannah_tracks_and_genres.shape[0]

In [65]:
for genre in genres_list:
    hannah_tracks_and_genres[genre] = 0
    for i in range(num_tracks):
        if genre in hannah_tracks_and_genres['genres'][i]:
            hannah_tracks_and_genres[genre][i] += 1
        else:
            pass

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [66]:
hannah_tracks_and_genres.shape

(3300, 512)

In [67]:
track_genre_faux_dummies = hannah_tracks_and_genres.drop(['artist_id', 'title', 'artist', 'album', 'album_id', 'time_added', 'genres', ''], axis=1).set_index('track_id')

In [68]:
track_genre_faux_dummies.head()

,album_releasedate,popularity,full_time_added,year_added,month_added,dayofweek_added,hour_added,danceability,energy,key,...,washington indie,welsh indie,welsh rock,west coast rap,wisconsin indie,wonky,world,world fusion,yacht rock,zapstep
track_id,,,,,,,,,,,,,,,,,,,,,
0jt6NEuzEY0OouXkwZHq5n,2006-01-01,44,2020-06-09 21:02:29,2020,6,1,21,0.584,0.967,11,...,0,0,0,0,0,0,0,0,0,0
1GAHIJ5a6AFbIW5yjrGdAz,2018-01-19,30,2020-06-09 07:50:30,2020,6,1,7,0.567,0.392,1,...,0,0,0,0,0,0,0,0,0,0
0QrxeCKmq1qHJSqysK5rdD,2016-01-29,35,2020-06-07 06:45:48,2020,6,6,6,0.544,0.539,0,...,0,0,0,0,0,0,0,0,0,0
1oG2vWELiGjIqxwqGcyqwF,1964-10-02,4,2020-06-06 21:30:23,2020,6,5,21,0.567,0.959,8,...,0,0,0,0,0,0,0,0,0,0
5fatsgkWJ3KcESzlObGiSL,2007-12-11,0,2015-07-08 03:25:02,2015,7,2,3,0.491,0.887,9,...,0,0,0,0,0,0,0,0,0,0


In [71]:
top_50_genres_artists = genres_df.sum().sort_values(ascending=False).head(50)

top_50_genres_artists.plot.barh(figsize=(5,15)).invert_yaxis();

# for other chart values see https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html#pandas.DataFrame.plot

NameError: name 'genres_df' is not defined

In [ ]:
top_50_genres_tracks = track_genre_faux_dummies.sum().sort_values(ascending=False).head(50)

top_50_genres_tracks.plot.barh(figsize=(5,15)).invert_yaxis();

# for other chart values see https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html#pandas.DataFrame.plot

In [ ]:
list(likedtracks_info_audiofeatures.columns)

In [ ]:
features_for_merge = likedtracks_info_audiofeatures.drop(['full_time_added','uri','track_href','analysis_url'], axis=1)

In [ ]:
likedtracks_info_audiofeatures.shape

In [ ]:
track_genre_faux_dummies.head(1)

In [ ]:
track_genre_faux_dummies.shape

In [ ]:
hannah_tracks_and_genres_dummies = features_for_merge.merge(track_genre_faux_dummies, on='track_id') #lsuffix='_tracks', rsuffix='_genres'

In [ ]:
hannah_tracks_and_genres_dummies.shape

In [ ]:
hannah_tracks_and_genres_dummies.dtypes.head(20)

In [ ]:
hannah_tracks_and_genres_dummies.to_csv('01-Data--My-Library/tracks_and_genres_dummies.csv')

In [ ]:
# genres_df['genres'].mask('Progressive Bluegrass' in genres_df['genres'])

In [ ]:
# count = 0

# for i in range(559):
#     if 'progressive bluegrass' in genres_df['genres'][i]:
        
#         print(i)
    
# print(count)

In [ ]:
# genres_df.head()

In [ ]:
# for track in genres_df['genres']:
#     for genre in genres_list:
#         if genre in track:
#             print(track)

In [ ]:
# for i in test_list: 
#     if(i == 4) : 
#         print ("Element Exists") 